# Classification Model Training

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from sklearn.model_selection import train_test_split
from PIL import Image

from tensorflow import keras
from tensorflow.keras import layers,models
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import Callback, EarlyStopping,ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import Model
from tensorflow.keras.layers.experimental import preprocessing

from pathlib import Path
import os.path

from sklearn.metrics import classification_report, confusion_matrix
import itertools
import torch

In [2]:
BATCH_SIZE = 32
IMAGE_SIZE = (320, 320)

In [3]:
dataset = "./DeepFire Dataset"

In [4]:
image_dir = Path(dataset)

# Get filepaths and labels
filepaths = list(image_dir.glob(r'**/*.JPG')) + list(image_dir.glob(r'**/*.jpg')) + list(image_dir.glob(r'**/*.png'))

labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

# Concatenate filepaths and labels
image_df = pd.concat([filepaths, labels], axis=1)

In [5]:
len(list(image_dir.glob(r'**/*.jpg')))

1900

In [6]:
# Separate in train and test data
train_df, test_df = train_test_split(image_df, test_size=0.3, shuffle=True, random_state=42)

In [7]:
train_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.3
)

test_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [8]:
# Split the data into three categories.
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

Found 931 validated image filenames belonging to 2 classes.
Found 570 validated image filenames belonging to 2 classes.
Found 399 validated image filenames belonging to 2 classes.


In [9]:
# Resize the Layer
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(224,224),
  layers.experimental.preprocessing.Rescaling(1./255),
])

Metal device set to: Apple M1


2024-04-07 13:43:30.881550: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-04-07 13:43:30.882184: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
# Load the Pre-Trained MobileNetV2 Model
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)
pretrained_model.trainable = False

In [11]:
# Create a Checkpoint Callback
checkpoint_path = "fires_classification_model_checkpoint"
checkpoint_callback = ModelCheckpoint(checkpoint_path,
                                      save_weights_only=True,
                                      monitor="val_accuracy",
                                      save_best_only=True)

In [12]:
# Setup EarlyStopping callback to stop training if the model's val_loss doesn't improve for 5 epochs
early_stopping = EarlyStopping(monitor="val_loss",
                               patience=5,
                               restore_best_weights=True)

In [13]:
# Add a new Fully Connected layer for binary classification
inputs = pretrained_model.input

x = pretrained_model.output
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)

# Add a Fully Connected layer for binary classification (Fire or Non-Fire)
outputs = Dense(2, activation='sigmoid')(x)
model = Model(inputs=inputs, outputs=outputs)
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [14]:
# model.save("mobilenetv2.keras")
import keras
model = keras.models.load_model("model.keras")

In [15]:
results = model.evaluate(test_images, verbose=0)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

2024-04-07 13:43:35.546003: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-04-07 13:43:35.790648: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


    Test Loss: 0.03798
Test Accuracy: 98.77%


# Building The Pipeline

In [16]:
# import dependencies
import os, re, time, json
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from functools import reduce
from sklearn.datasets import make_blobs

In [17]:
IMG_HEIGHT = 512
IMG_WIDTH = 512

In [18]:
def get_image_feature(image):
    temp = image # divide by 255 to get in fraction
    mn = temp.sum(axis=0).sum(axis=0)/(temp.shape[0]*temp.shape[1])
    return mn/np.linalg.norm(mn, ord=None) # taking 2nd norm to scale vector

def get_image(name, folder, color_space='rgb'):
    filepath = os.path.join(folder, name)
    # img = Image.open(filepath)
    # read imread as rgb
    img = cv2.imread(filepath, cv2.IMREAD_COLOR)
    # convert to rgb color space
    if color_space == 'rgb':
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    elif color_space == 'gray':
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # resize image to 254x254
    img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH))
    # convert to rgb color space
    return np.array(img)


def get_images(directoryName, color_space='rgb'):
    directory = os.fsencode(directoryName)
    images = []
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
            image = get_image(filename, directoryName, color_space)
            images.append(image)
            continue
        else:
            continue
    return np.array(images)

In [19]:
from sklearn.mixture import GaussianMixture

def low_light(image):
    # Convert to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Apply histogram equalization to V channel
    # hsv[:, :, 2] = cv2.equalizeHist(hsv[:, :, 2])

    # # Apply median filter to V channel
    # hsv[:, :, 2] = cv2.medianBlur(hsv[:, :, 2], 5)

    # Convert back to RGB
    img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    return img

def apply_hsv_rule(image):
    img = image.copy()

    # Convert to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

    # Split channels
    h, s, v = cv2.split(hsv)

    # transform hsv to 0-1 range
    h = h/255
    s = s/255
    v = v/255

    # get rgb component
    r, g, b = cv2.split(image)

    # Define rules
    mask = (h>=0.13) & (h<0.46) & (s>=0.1) & (s<=0.34) & (v>=0.96) & (v<=1) & (r>=g) & (g>b) & (r>180) & (g>130) & (b<120)

    # Apply mask
    img[mask] = 255

    img[~mask] = 0

    return img

def apply_ycbcr_rule(image):
    img = image.copy()

    # Convert to YCbCr
    ycbcr = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)

    # Split channels
    y, cb, cr = cv2.split(ycbcr)

    # get rgb component
    r, g, b = cv2.split(image)

    # Define rules
    mask = (y>cr) & (cr>=cb) & (y>=180) & (y<210) & (cb>=80) & (cb<=120) & (cr>=80) & (cr<=139) & (r>=g) & (g>b) & (r>190) & (g>110) & (b<180)

    # Apply mask
    img[mask] = 255

    img[~mask] = 0

    return img

def apply_rgb_rule(image):
    img = image.copy()

    # Split channels
    r, g, b = cv2.split(image)

    # Define rules
    mask = (r>g) & (g>b) & (r>200) & (g>130) & (b<120)

    # Apply mask
    img[mask] = 255

    img[~mask] = 0

    return img

def segment_images(images):
    result = []
    for image in images:
        cpy_img = np.copy(image)

        # apply low light
        cpy_img = low_light(cpy_img)

        # apply hsv rules
        ycbcr_img = apply_ycbcr_rule(cpy_img)
        hsv_img = apply_hsv_rule(cpy_img)
        rgb_img = apply_rgb_rule(cpy_img)

        # combine all masks using or operator
        ycbcr_img = np.logical_or(ycbcr_img, hsv_img)
        ycbcr_img = np.logical_or(ycbcr_img, rgb_img)

        # mask original image
        cpy_img[~ycbcr_img] = 0
        cpy_img[ycbcr_img] = 255

        result.append(cpy_img)

    return np.array(result)

def segment_image(image):
    cpy_img = np.copy(image)

    # apply low light
    cpy_img = low_light(cpy_img)

    # apply hsv rules
    ycbcr_img = apply_ycbcr_rule(cpy_img)
    hsv_img = apply_hsv_rule(cpy_img)
    rgb_img = apply_rgb_rule(cpy_img)

    # combine all masks using or operator
    ycbcr_img = np.logical_or(ycbcr_img, hsv_img)
    ycbcr_img = np.logical_or(ycbcr_img, rgb_img)

    # mask original image
    cpy_img[~ycbcr_img] = 0

    return cpy_img


In [20]:
def morphological_operations(image, kernel_size, iterations):
    # Create a kernel for morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size)

    # Perform closing operation
    closed_image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel, iterations=iterations)

    # Perform dilation operation
    dilated_image = cv2.dilate(closed_image, kernel, iterations=iterations)

    # close holes
    # dilated_image = cv2.morphologyEx(dilated_image, cv2.MORPH_CLOSE, kernel, iterations=iterations)


    return dilated_image

In [21]:
def get_prediction(image):
    masked = segment_image(image)

    masked = cv2.cvtColor(masked, cv2.COLOR_RGB2GRAY)

    masked = morphological_operations(masked, (5, 5), 4)

    return masked

In [22]:
test_image = cv2.imread("./FireSegmentationDataset/Video01/Video01_Frame/Video01_Frame001.jpg")
test_image_reshaped = np.resize(test_image,(1, 224, 224, 3))

In [23]:
test_image_reshaped.shape

(1, 224, 224, 3)

In [24]:
def evaluate(image):
    pred = model.predict(image)
    mask = get_prediction(image)
    return mask

In [26]:
%%time
pred = model.predict(test_image_reshaped)
mask = get_prediction(test_image_reshaped[0])

1/1 [==============================] - 0s 36ms/step
CPU times: user 72.3 ms, sys: 22.5 ms, total: 94.8 ms
Wall time: 87.4 ms


# Diki

In [27]:
from ultralytics import YOLO

In [28]:
modelv8_1 = YOLO('./fire-segmentation-master/models/bestv8_1.pt')
modelv8_2 = YOLO('./fire-segmentation-master/models/bestv8_2.pt')

modelv8 = [modelv8_1, modelv8_2]

In [29]:
def low_light(image):
    # Convert to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Apply histogram equalization to V channel
    hsv[:, :, 2] = cv2.equalizeHist(hsv[:, :, 2])

    # Apply median filter to V channel
    hsv[:, :, 2] = cv2.medianBlur(hsv[:, :, 2], 5)

    # Convert back to RGB
    img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    return img

def ycbcr(image):
    img = image.copy()

    # Convert to YCbCr
    ycbcr = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

    # Split channels
    y, cb, cr = cv2.split(ycbcr)

    # Define rules
    mask = (y >= 170) | (y < 145) & (cb <= 120) & (cb >= 50) & (cr > 120) & (cr < 220)

    # Apply mask
    img[mask] = 255

    img[~mask] = 0

    return img

def rgb_space(image):
    img = image.copy()

    # Split channels
    b, g, r = cv2.split(image)

    # Define rules
    rules = [
        r > g,
        g > b,
        r > 190,
        g > 90,
        b < 140,
        0.1 <= (g/(r+1)),
        (g/(r+1)) <= 1,
        0.1 <= (b/(r+1)),
        (b/(r+1)) <= 0.85,
        0.1 <= (b/(g+1)),
        (b/(g+1)) <= 0.85
    ]

    # Apply rules
    mask = np.all(rules, axis=0)

    # Apply mask
    img[mask] = 255

    img[~mask] = 0

    return img

# Get percentage from 2 images spatially
def get_percentage(img1, img2):
    # Convert to grayscale
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # Get percentage of white pixels location
    percentage = np.sum(img1 == img2) / (img1.shape[0] * img1.shape[1])

    return percentage

def segment(img):
    # Enhance image
    low_light_img = low_light(img)

    # Apply RGB space segmentation
    rgb_space_img = rgb_space(img)
    rgb_space_low_light = rgb_space(low_light_img)

    # Apply YCbCr segmentation
    ycbcr_img = ycbcr(img)
    ycbcr_low_light = ycbcr(low_light_img)

    # Combine RGB space and YCbCr segmented images using bitwise OR for both original and low light images
    combined_img = cv2.bitwise_or(rgb_space_img, ycbcr_img) 
    combined_img_low = cv2.bitwise_or(rgb_space_low_light, ycbcr_low_light)

    # Combine segmented original and low light images using bitwise AND
    ultimate_combined = cv2.bitwise_and(combined_img, combined_img_low)

    # Combine segmented RGB original and low light images using bitwise AND
    rgb_combined = cv2.bitwise_or(rgb_space_img, rgb_space_low_light)

    # Filter the result based on the threshold
    output = np.zeros_like(ultimate_combined)

    # Get the percentage of white pixels spatially
    percentage = get_percentage(ultimate_combined, rgb_combined)

    if percentage >= 0.75:
        output = rgb_combined
    else:
        output = ultimate_combined

    # Convert to grayscale
    output = cv2.cvtColor(output, cv2.COLOR_BGR2GRAY)

    return output

# Filling holes in the segmented image
def fill_holes(image):
    # Perform morphological closing operation to fill small holes
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4, 4))

    # Perform closing operation
    closed_image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)

    # Perform dilation operation
    filled_image = cv2.dilate(closed_image, kernel, iterations=2)

    return filled_image

# Crop the image from the original image
def crop_image(image, x1, y1, x2, y2):
    return image[y1:y2, x1:x2]

def detect(image_path):
    # Create output folder if not exists
    if not os.path.exists('output'):
        os.makedirs('output')

    # Define coordinates
    coordinates = []

    ## YOLOv8
    # Detecting objects in an image with YOLOv8
    for model in modelv8:
        results = model.predict(image_path)

        # YOLOv8 coordinates
        for r in results:
            boxes = r.boxes
            for box in boxes:
                # print(box)
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                # print(x1, y1, x2, y2)
                coordinates.append([x1, y1, x2, y2])

    image = cv2.imread(image_path)
    cv2.imwrite('./output/original.jpg', image)

    # Initialize the binary image
    binary_image = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

    # Iterate through the coordinates
    for coordinate in coordinates:
        # Get coordinates
        x1, y1, x2, y2 = coordinate

        # Crop the image
        cropped_image = crop_image(image, x1, y1, x2, y2)

        # Segment the cropped image
        segmented_image = segment(cropped_image)

        # Fill holes in the segmented image
        filled_image = fill_holes(segmented_image)

        # Add the segmented image to the binary image
        binary_image[y1:y2, x1:x2] = binary_image[y1:y2, x1:x2] + filled_image

    cv2.imwrite('./output/binary.jpg', binary_image)
    return binary_image

In [31]:
%%time
binary_image = detect('./FireSegmentationDataset/Video01/Video01_Frame/Video01_Frame001.jpg')


image 1/1 /Users/widss/tensorflow-test/Efficient-Wildfire-Detection-Framework-Based-on-AI-Using-CNN-and-Multi-Color-Filtering-main/FireSegmentationDataset/Video01/Video01_Frame/Video01_Frame001.jpg: 416x640 (no detections), 80.8ms
Speed: 3.5ms preprocess, 80.8ms inference, 0.4ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 /Users/widss/tensorflow-test/Efficient-Wildfire-Detection-Framework-Based-on-AI-Using-CNN-and-Multi-Color-Filtering-main/FireSegmentationDataset/Video01/Video01_Frame/Video01_Frame001.jpg: 192x256 2 fires, 14.6ms
Speed: 0.4ms preprocess, 14.6ms inference, 0.3ms postprocess per image at shape (1, 3, 192, 256)
CPU times: user 169 ms, sys: 54.1 ms, total: 223 ms
Wall time: 115 ms


/var/folders/kp/1yj5j5nx6gddj5pqbjbnl45r0000gn/T/ipykernel_20129/1968910619.py:48: RuntimeWarning: divide by zero encountered in divide
  0.1 <= (g/(r+1)),
/var/folders/kp/1yj5j5nx6gddj5pqbjbnl45r0000gn/T/ipykernel_20129/1968910619.py:49: RuntimeWarning: divide by zero encountered in divide
  (g/(r+1)) <= 1,
/var/folders/kp/1yj5j5nx6gddj5pqbjbnl45r0000gn/T/ipykernel_20129/1968910619.py:50: RuntimeWarning: divide by zero encountered in divide
  0.1 <= (b/(r+1)),
/var/folders/kp/1yj5j5nx6gddj5pqbjbnl45r0000gn/T/ipykernel_20129/1968910619.py:51: RuntimeWarning: divide by zero encountered in divide
  (b/(r+1)) <= 0.85,
/var/folders/kp/1yj5j5nx6gddj5pqbjbnl45r0000gn/T/ipykernel_20129/1968910619.py:52: RuntimeWarning: divide by zero encountered in divide
  0.1 <= (b/(g+1)),
/var/folders/kp/1yj5j5nx6gddj5pqbjbnl45r0000gn/T/ipykernel_20129/1968910619.py:53: RuntimeWarning: divide by zero encountered in divide
  (b/(g+1)) <= 0.85


# cv bani

In [52]:
!git clone https://github.com/whyspaceee/fire-segmentation.git

Cloning into 'fire-segmentation'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 27 (delta 4), reused 5 (delta 1), pack-reused 0
Receiving objects: 100% (27/27), 43.13 MiB | 8.03 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [32]:
import tensorflow_advanced_segmentation_models as tasm

from tensorflow import keras

In [33]:
  BATCH_SIZE = 16
  BUFFER_SIZE = 1000
  N_CLASSES = 2
  HEIGHT = 224
  WIDTH = 224
  BACKBONE_NAME = "efficientnetb3"
  WEIGHTS = "imagenet"

In [34]:
base_model, layers, layer_names = tasm.create_base_model(name=BACKBONE_NAME, weights=WEIGHTS, height=HEIGHT, width=WIDTH, include_top=False, pooling=None)

BACKBONE_TRAINABLE = False 
model = tasm.DeepLabV3plus(n_classes=N_CLASSES, base_model=base_model, output_layers=layers, backbone_trainable=BACKBONE_TRAINABLE)

In [35]:
model.load_weights('./fire-segmentation/weights/DeepLab224')

In [36]:
detector = tf.keras.models.load_model('./fire-segmentation/detection-model-mobilenet224-v3.h5')

In [37]:
def create_mask(pred_mask):
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    pred_mask = tf.cast(pred_mask, tf.float32)
    return pred_mask[0]

In [38]:
def display(display_list):
    plt.figure(figsize=(15, 15))

    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i + 1)
        plt.title(title[i])

        # Convert mask to grayscale
        if i == 1 or i == 2:
            mask = np.array(display_list[i])
            mask = np.squeeze(mask, axis=-1)
            plt.imshow(mask, cmap='gray')
        else:
            plt.imshow(display_list[i])

        plt.axis('off')
    plt.show()

In [39]:
def preprocess_image(image):
    # Convert image from RGB to HSV color space
    hsv_image = cv2.cvtColor(reshaped_image, cv2.COLOR_RGB2HSV)
    # Apply histogram equalization to the V channel of the HSV image
    hsv_image[:, :, 2] = cv2.equalizeHist(hsv_image[:, :, 2])
    # Perform median filtering on the V channel
    filtered_image = cv2.medianBlur(hsv_image[:, :, 2], 5)
    # Convert the HSV image back to RGB color space
    processed_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2RGB)
    return processed_image, filtered_image


def extract_flames_rgb(image):
    # Extract flames based on pixel distribution rules in RGB color space
    red_channel = image[:, :, 0]
    green_channel = image[:, :, 1]
    flames_rgb = np.logical_and(red_channel > green_channel, red_channel > 177)
    return flames_rgb


def extract_flames_ycbcr(image):
    # Convert RGB image to YCbCr color space
    ycbcr_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    # Extract flames based on pixel distribution rules in YCbCr color space
    y_channel = ycbcr_image[:, :, 0]
    cb_channel = ycbcr_image[:, :, 1]
    cr_channel = ycbcr_image[:, :, 2]
    cb_threshold = np.percentile(cb_channel, 45)
    cr_threshold = np.percentile(cr_channel, 70)
    flames_ycbcr = np.logical_and.reduce([
        y_channel > 75,
        cb_channel < cb_threshold,
        cr_channel > cr_threshold
    ])
    return flames_ycbcr


def apply_and_operator(flames_rgb, flames_ycbcr):
    # Apply 'and' operator to obtain a complete flame image
    flames = np.logical_and(flames_rgb, flames_ycbcr)
    return flames


def apply_or_operator(flames_rgb, flames_ycbcr):
    # Apply 'and' operator to obtain a complete flame image
    flames = np.logical_or(flames_rgb, flames_ycbcr)
    return flames


# Load the original flame image
def segment_img(img):
    mask = create_mask(
        model.predict(np.asarray(img)[tf.newaxis, ...])
    )
    THRESHOLD = 0.8
    pred = detector.predict(np.asarray(img)[tf.newaxis, ...])
    if pred[0][0] > THRESHOLD:
        return np.zeros((224, 224, 1))
    processed_image, filtered_image = preprocess_image(np.asarray(img))
    flames_rgb = extract_flames_rgb(processed_image)
    flames_ycbcr = extract_flames_ycbcr(processed_image)
    flames = apply_and_operator(flames_rgb, flames_ycbcr)
    flamesImg = tf.keras.preprocessing.image.array_to_img(
        (flames.astype(np.uint8) * 255)[:, :, np.newaxis])
    
    filtered = (flames.astype(np.uint8) * 255)[:, :, np.newaxis]
    seg = apply_and_operator(filtered, mask)
    return seg

In [40]:
tf.config.run_functions_eagerly(True)

In [42]:
%%time
segment_img(test_image_reshaped[0])

1/1 [==============================] - 0s 95ms/step
CPU times: user 299 ms, sys: 94.2 ms, total: 394 ms
Wall time: 453 ms


array([[[          0],
        [          0],
        [          0],
        ...,
        [          0],
        [          0],
        [          0]],

       [[          0],
        [          0],
        [          0],
        ...,
        [          0],
        [          0],
        [          0]],

       [[          0],
        [          0],
        [          0],
        ...,
        [          0],
        [          0],
        [          0]],

       ...,

       [[          0],
        [          0],
        [          0],
        ...,
        [          0],
        [          0],
        [          0]],

       [[          0],
        [          0],
        [          0],
        ...,
        [          0],
        [          0],
        [          0]],

       [[          0],
        [          0],
        [          0],
        ...,
        [          0],
        [          0],
        [          0]]])

In [46]:
detector.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 MobilenetV3large (Functiona  (None, 7, 7, 960)        2996352   
 l)                                                              
                                                                 
 global_average_pooling2d (G  (None, 960)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 64)                61504     
                                                                 
 dense_1 (Dense)             (None, 16)                1040      
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 3,058,913
Trainable params: 62,561
Non-tra